# IS 590 (Data Visualization)
# Group_Name: Oli-zza

## Group_Member:

Chunzi Pan; Kaiyang Hu; Keith Hollenkamp; Yi Sun;

### Read in data

This part is to read in data and install packages for part 1

In [1]:
%matplotlib inline

In [2]:
import matplotlib.cm as cm
from matplotlib.colors import LogNorm, PowerNorm
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
import ipywidgets
import cartopy
import csv
import us
from IPython.display import Audio
from mpl_toolkits.axes_grid1 import make_axes_locatable
from haversine import haversine
import dateutil.parser as parser

states = cartopy.io.shapereader.natural_earth(resolution = '110m', category = 'cultural', name = 'admin_1_states_provinces_lakes_shp')
reader = cartopy.io.shapereader.Reader(states)
all_states = list(reader.records())
geometries = [state.geometry for state in all_states]
array_vals=pd.read_csv("/srv/nbgrader/data/transportable_array/data_tohoku_norm_transpose.csv", header=None)

In [3]:
locations=pd.read_csv("/srv/nbgrader/data/transportable_array/location.txt",
                      delimiter="\t", names=["longitude","latitude","a","b"])
del locations["a"], locations["b"] # delete the final two columns

### View data

In [4]:
array_vals.head(5)

,0,1,2,3,4,5,6,7,8,9,...,428,429,430,431,432,433,434,435,436,437
0,-0.000255,-0.000068,0.000406,-0.000275,0.000070,0.000041,-0.000070,-0.000040,0.000152,0.000039,...,-0.000156,0.000025,0.000064,0.000047,0.000030,0.000080,0.000051,0.000156,0.000282,-0.000024
1,-0.000238,-0.000085,0.000411,-0.000293,0.000085,0.000058,-0.000027,-0.000042,0.000141,0.000151,...,-0.000118,0.000127,0.000029,0.000056,0.000093,0.000016,0.000016,0.000056,0.000280,-0.000036
2,-0.000220,-0.000102,0.000413,-0.000308,0.000099,0.000073,0.000015,-0.000042,0.000129,0.000259,...,-0.000080,0.000225,-0.000005,0.000065,0.000153,-0.000046,-0.000019,-0.000042,0.000275,-0.000049
3,-0.000203,-0.000117,0.000411,-0.000321,0.000111,0.000087,0.000055,-0.000042,0.000116,0.000360,...,-0.000043,0.000316,-0.000037,0.000073,0.000208,-0.000106,-0.000052,-0.000135,0.000269,-0.000060
4,-0.000186,-0.000130,0.000406,-0.000330,0.000122,0.000099,0.000092,-0.000041,0.000103,0.000451,...,-0.000009,0.000398,-0.000067,0.000080,0.000257,-0.000160,-0.000083,-0.000220,0.000261,-0.000071


In [5]:
locations.head(5)

,longitude,latitude
0,-98.683,27.065
1,-98.102,26.938
2,-98.068,26.463
3,-117.110,32.889
4,-107.790,32.532


## Part1

######  Data for spectrogram

In [ ]:
# Prepare data
array_vals_sorted=np.zeros((14401,438))

Find the latitude and longitude of tohoku location from https://earthquake.usgs.gov/earthquakes/eventpage/official20110311054624120_30#executive

In [ ]:
tohoku_loc=(142.373, 38.297)

In [6]:
distance=[]
for i in range(0,437):
    x=(locations['longitude'][i],locations['latitude'][i])
    distance.append(haversine(tohoku_loc,x, miles=True))
# distance
# sort the distance #ds_sort=sorted(distance)

ds_indicator=np.argsort(distance) # get the sorted distance number
for i in range(0,437):
    array_vals_sorted[:,i]=array_vals[ds_indicator[i]]

matrix_vals = np.asmatrix(array_vals_sorted)5
matrix_vals = matrix_vals.transpose()

######  Three plots for Part 1

In [7]:
##Combination 
@ipywidgets.interact(stat_id = (0,437,1), time = (0, 14400, 1))
def make_total(stat_id = 0, time = 0):
    
    fig = plt.figure(figsize=(10,10))
    
    # Map
    axes_1 = fig.add_subplot(211, projection = cartopy.crs.LambertConformal())
    axes_1.set_extent([-125, -66.5, 20, 50], cartopy.crs.Geodetic())
    axes_1.add_geometries(geometries, cartopy.crs.PlateCarree(), edgecolors = 'white', facecolor = 'pink', zorder = 0)
    mappable = axes_1.scatter(locations['longitude'], locations['latitude'],
                              transform = cartopy.crs.Geodetic(), 
                              c = (array_vals.iloc[time]),
                              cmap = "RdYlBu")
    
    axes_1.set_title("Stations in the Transportable Array")
    fig.suptitle("Measurements of the Tohoku Earthquake", fontsize = 20)
    
    plt.colorbar(mappable,fraction=0.03, pad=0.01, label = "Intensity of Earthquake")
     
    # Spectrogram   
    axes_2 = plt.subplot(223)
    plt.imshow(matrix_vals, aspect='auto',cmap='jet', origin='lower')
    plt.xlabel('Time in Seconds')
    plt.ylabel('American Station')
    
    ax21 = plt.axvline(0, color = 'r')
    x21, y21 = ax21.get_data()
    ax21.set_data(([a+ time for a in x21],y21 ))
    
    ax22 = plt.axhline(0, color = 'r')
    x22, y22 = ax22.get_data()
    ax22.set_data((x22,[a+stat_id for a in y22] ))
    
    # Line plot
    axes_3 = plt.subplot(224)
    plt.plot(array_vals[stat_id])
    plt.xlabel("Time in Seconds")
    plt.ylabel("Earthquake Intensity", labelpad = -3)

interactive(children=(IntSlider(value=0, description='stat_id', max=437), IntSlider(value=0, description='time', max=14400), Output()), _dom_classes=('widget-interact',))

## Part 3

###### Make data for part 3

In [ ]:
from datetime import datetime
import bqplot
from bqplot import (
   Figure, Map, Mercator, Orthographic, ColorScale, ColorAxis,
   AlbersUSA, topo_load, Tooltip
)
from ipywidgets import ToggleButtons, VBox, HTML, Dropdown,interact
from copy import deepcopy

In [8]:
names = ["date", "city", "state", "country", "shape", "duration_seconds",
         "duration_reported", "description", "report_date", "latitude",
         "longitude"]

fn = "/srv/nbgrader/data/ufo-scrubbed-geocoded-time-standardized.csv"

In [9]:
ufo = pd.read_csv(fn, names = names, parse_dates = ["date", "report_date"])

In [10]:
abbr_to_fits = us.states.mapping('abbr', 'fips')
ufo["fips"] = ufo['state'].apply(lambda a: int(abbr_to_fits.get(str(a).upper(), -1)))

total_time=ufo.groupby('state')['duration_seconds'].sum()
total_sightings = ufo.groupby("state")["report_date"].count()

In [11]:
df = pd.DataFrame([total_time, total_sightings])
df.rename({'duration_seconds': 'total_time'}, inplace=True)
df.rename({'report_date': 'total_sightings'}, inplace=True)
ufo_vals = df.transpose()

# Extract year from date

In [12]:
report_year=[]
for i in range(0,80331):
    report_year.append(parser.parse("{:%B %d, %Y}".format(ufo["date"][i])).year)

In [13]:
year=pd.DataFrame(report_year,columns=['year'])
ufo.insert(loc=0, column='year', value=year) 

In [38]:
ufo.head()

,year,date,city,state,country,shape,duration_seconds,duration_reported,description,report_date,latitude,longitude,fips
0,1949.0,1949-10-10 20:30:00,san marcos,tx,us,cylinder,2700.0,45 minutes,This event took place in early fall around 194...,2004-04-27,29.883056,-97.941111,48
1,1949.0,1949-10-10 21:00:00,lackland afb,tx,NaN,light,7200.0,1-2 hrs,1949 Lackland AFB&#44 TX. Lights racing acros...,2005-12-16,29.384210,-98.581082,48
2,1955.0,1955-10-10 17:00:00,chester (uk/england),NaN,gb,circle,20.0,20 seconds,Green/Orange circular disc over Chester&#44 En...,2008-01-21,53.200000,-2.916667,-1
3,1956.0,1956-10-10 21:00:00,edna,tx,us,circle,20.0,1/2 hour,My older brother and twin sister were leaving ...,2004-01-17,28.978333,-96.645833,48
4,1960.0,1960-10-10 20:00:00,kaneohe,hi,us,light,900.0,15 minutes,AS a Marine 1st Lt. flying an FJ4B fighter/att...,2004-01-22,21.418056,-157.803611,15


## BQplot

create pivot table

In [15]:
pivo1 = pd.pivot_table(ufo,index=["state"], values=["fips"], fill_value=0)
pivo2 = pd.pivot_table(ufo,index=["state"], values=["duration_seconds", "latitude"], 
               aggfunc={"duration_seconds": np.sum, "latitude": (lambda x: len(x))}, 
               fill_value=0, margins= True)

ufo_vals = pd.merge(pivo1, pivo2, left_index=True, right_index=True)
ufo_vals.rename(index=str, columns={'duration_seconds': 'total_time', 'latitude': 'total_sightings', 'fips': 'fips'},inplace = True)

In [16]:
pivo3 = pd.pivot_table(ufo, index=["state"], values=["duration_seconds", "latitude"], 
                       columns = ['year'],
                       aggfunc={"duration_seconds": np.sum, "latitude": (lambda x: len(x))}, 
                       fill_value=0)
pivo3.rename(index=str, columns={'duration_seconds': 'total_time', 'latitude': 'total_sightings'},inplace = True)

Make data for normalization
We used a cencus data from https://data.world/aaronhoffman/census-gov-state-quickfacts.
A separate cleaned csv file (State_Pop_Cleaned.csv) is uploaded with this notebook on GitHub.

In [26]:
pop = pd.read_csv('State_Pop_Cleaned.csv')

In [36]:
pop.head()

,State,Population,FIPS
0,Alaska,741894,"""02"""
1,Georgia,10310371,"""13"""
2,Hawaii,1428557,"""15"""
3,Idaho,1683140,"""16"""
4,Illinois,12801539,"""17"""


In [19]:

ufo_pop = ufo_vals[ufo_vals['fips'] != -1]
ufo_pop = ufo_pop[ufo_pop.index.values != 'pr'] # Puerto Rico 

for i in range(len(pop['FIPS'])):
    pop['FIPS'][i] = pop['FIPS'][i].lstrip('"')
    pop['FIPS'][i] = pop['FIPS'][i].rstrip('"')

tmp = pop.sort_values(by=['FIPS'])[['Population']]
ufo_pop_test = ufo_pop.sort_values(by = ['fips'])
tmp.index = range(51)
ufo_pop_test.index = range(51)
ufo_pop_test.insert(loc = 0, column = 'Population', value = tmp)

ufo_pop_test['norm_total_time'] = ufo_pop_test['total_time']/ufo_pop_test['Population']
ufo_pop_test['norm_total_sightings'] = ufo_pop_test['total_sightings']/ufo_pop_test['Population']

/opt/conda/lib/python3.6/site-packages/ipykernel_launcher.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  
/opt/conda/lib/python3.6/site-packages/ipykernel_launcher.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  import sys


In [29]:
ufo_pop_test.head(5)

,Population,fips,total_time,total_sightings,norm_total_time,norm_total_sightings
0,4863300,1,962845.50,691,0.197982,0.000142
1,741894,2,1498068.00,354,2.019248,0.000477
2,6931071,4,15990939.10,2689,2.307138,0.000388
3,2988248,5,67177514.50,666,22.480569,0.000223
4,39250017,6,37928460.47,9655,0.966330,0.000246


In [32]:
ufo_final = deepcopy(ufo_vals)
ufo_final.index = ufo_vals['fips']
ufo_final2 = deepcopy(ufo_vals)

In [21]:
ufo_final2 = ufo_final2[ufo_final2['fips']!=-1]
ufo_final2 = ufo_final2[ufo_final2.index.values != 'pr']
ufo_final2 = ufo_final2.sort_values(by = ['fips'])
ufo_final2.index =  range(51)
insert_vals = ufo_pop_test.loc[:, ['norm_total_time','norm_total_sightings']]

In [22]:
ufo_final2 = ufo_final2[['fips']].join(insert_vals)

ufo_final2.index = ufo_final2['fips']

In [23]:
ufo_final = ufo_final[ufo_final['fips']!=-1]
ufo_final = ufo_final[ufo_final['fips']!=72]
ufo_final = ufo_final.sort_values(by = 'fips')

In [24]:
ufo_final3 = ufo_final2.join(ufo_final.loc[:,['total_time','total_sightings']])

In [34]:
ufo_final3.head(5)

,fips,norm_total_time,norm_total_sightings,total_time,total_sightings
fips,,,,,
1,1,0.197982,0.000142,962845.50,691
2,2,2.019248,0.000477,1498068.00,354
4,4,2.307138,0.000388,15990939.10,2689
5,5,22.480569,0.000223,67177514.50,666
6,6,0.966330,0.000246,37928460.47,9655


Here we delete places whose fips == -1 and 72 is because these places are out of US map

## BQplot for Part 3

In [37]:
pivo3.head()

total_time                                                          \
year      1910.0 1920.0 1925.0 1929.0 1930.0 1931.0 1933.0 1934.0 1936.0   
state                                                                      
ab             0      0      0      0      0      0      0      0      0   
ak             0      0      0      0      0      0      0      0    180   
al             0      0      0      0      0      0      0      0      0   
ar             0      0      0      0      0      0      0      0      0   
az             0      0      0      0      0      0      0      0      0   

              ...   total_sightings                                            \
year  1937.0  ...            2005.0 2006.0 2007.0 2008.0 2009.0 2010.0 2011.0   
state         ...                                                               
ab         0  ...                30     16     18     14     14     19     15   
ak         0  ...                20     13      6      9     16     13     17   
al         0  ...                38     36     44     43     30     25     37   
ar         0  ...                32     29     30     36     25     39     41   
az         0  ...               181    127    123    164    150    109    150   

                            
year  2012.0 2013.0 2014.0  
state                       
ab        17     20      8  
ak        43     45     24  
al        55     71     47  
ar        52     50      9  
az       194    215    109  

[5 rows x 170 columns]

In [25]:
@ipywidgets.interact(field = ['total_time','total_sightings','norm_total_sightings', 'norm_total_time'])
def make_plot(field):
    for fips in ufo_final3['fips']:
        n = ufo_final3.loc[:, field]
    def_tt = Tooltip(fields=['name', 'color'], labels=['State', field])
   
    map_styles = {'scales': {'projection': bqplot.AlbersUSA(),
                            'color': bqplot.ColorScale(colors=["pink","Purple"])},
                  'color': n.to_dict()}
   
    states_map = bqplot.Map(map_data=bqplot.topo_load('map_data/USStatesMap.json'), tooltip=def_tt,
        **map_styles)
    states_map.interactions = {'click': 'select', 'hover': 'tooltip'}
    map_fig = bqplot.Figure(marks=[states_map], title='USA', fig_margin={'top':0, 'bottom':0, 'left':0, 'right':0})
   
    x_sc = bqplot.LinearScale()
    y_sc = bqplot.LinearScale()
    # line plot
    if field in ['total_time','norm_total_time']:
        def print_figure(self, target):
            data1 = target
            data2 = data1['data']
            data = data2['id']
            values = pivo3[ufo_vals["fips"]==data]['total_time'].values[0]
            lineplot.x = list(map(int, pivo3['total_time'].columns))
            lineplot.y = values
        states_map.on_element_click(print_figure)
        lineplot = bqplot.Lines(x = [], y = [], scales = {'x': x_sc, 'y': y_sc})
        ax_x = bqplot.Axis(scale = x_sc, label = 'Year')
        ax_y = bqplot.Axis(scale = y_sc, orientation = "vertical", label = "Total Time(In Seconds)",
                           label_offset = '20')
        
        line_figure = bqplot.Figure(marks=[lineplot], axes = [ax_x, ax_y])
    else:
        def print_figure(self, target):
            data1 = target
            data2 = data1['data']
            data = data2['id']
            values = pivo3[ufo_vals["fips"]==data]['total_sightings'].values[0]
            lineplot.x = list(map(int, pivo3['total_time'].columns))
            lineplot.y = values
        states_map.on_element_click(print_figure)
 
        lineplot = bqplot.Lines(x = [], y = [], scales = {'x': x_sc, 'y': y_sc})

        ax_x = bqplot.Axis(scale = x_sc, label = 'Year')
        ax_y = bqplot.Axis(scale = y_sc, orientation = "vertical", label = "Sightings",
                           label_offset = '20')
        line_figure = bqplot.Figure(marks=[lineplot], axes = [ax_x, ax_y])

   
    display(map_fig, line_figure)

interactive(children=(Dropdown(description='field', options=('total_time', 'total_sightings', 'norm_total_sightings', 'norm_total_time'), value='total_time'), Output()), _dom_classes=('widget-interact',))